# 2. Solving Mastermind with information theory

Now we will code a "solution" to the game, a script that solves it. I used the ideas in: Solving Wordle using information theory. but with Mastermind instead of Wordle.

Here is the script that solves it.



In [60]:
# mastermind game playing using information thoery

import random
import math

''' 1. pick code and display it so we can see'''


def choose_code(max_number, code_length):
    
    Num_Total = list(range(1, max_number + 1))
    # this is the number of the numbers available to use.
    Code = ''
    
    for _ in range(code_length):
        Code += str(random.choice(Num_Total))
        # this randomly pick a number from the total available numbers\
    
    return Code


''' 2. Get clue based on the guess and the actual code'''

def Clues(code,guess):
    circle = 0
    check_mark = 0

    for i in range(len(code)):
        if code[i] == guess[i]:
            check_mark += 1
            code = code.replace(code[i], ' ',1)
            guess = guess.replace(guess[i], ' ',1)
            
    for number in code:
        if number ==' ':
            continue
        elif number in guess:
            circle += 1
            code = code.replace(number, ' ',1)
            guess = guess.replace(number, ' ',1)
            
    
    clue = (circle , check_mark)
    return clue


'''3. generates permutations of all the possible combinations. We use this to help solve the game '''


def generate_permutations_with_duplicates(numbers, length, current=''):
    n=''.join(str(i) for i in range(1, numbers + 1))
    if length == 0:
        return [tuple(current)]

    result = []
    for i in range(numbers):
        result.extend(generate_permutations_with_duplicates(numbers, length-1, current + n[i]))

    return result

'''4. get the probability distribution of a particular guess
      the way to do that is assume each code in the list is the real code and then see how many of each different clue is possible
      example: '1234' is the start_code and then we iterate through all the permutations of 1-6, to see how many of clues are possible
       '''

def generate_all_clues(n):
    counter = {}
    for i in range(n+1):
        for y in range(n+1):
            counter[(i,y)]= 0
    return counter


def get_prob_dist(start_code,permutations_list,n):
    counter = generate_all_clues(n)
    total = {}
    for thing in permutations_list:
        code = ''.join(thing)
        x = Clues(code,start_code)
        total[thing] = x
        counter[x] =counter[x]+1

    # print(counter)
    total = sum(counter.values())
    counter = {key: value / total for key, value in counter.items()}

    return counter

'''5. This calculates the amount of information for every code and the possible clues'''

def calculate_info(counter):
    information = {}
    for key,value in counter.items():
        if value != 0:
            information[key] = math.log2(1/value)
        else:
            information[key] = 0


    return information


'''6. This calculates the expected information or the entropy by summing up all the information * probabilities'''

def calculate_expected_information(probabilities):
    sum = 0
    for key,value in probabilities.items():
        if value != 0:
            sum = sum + (value * math.log2(1/value))
        else:
            sum = sum + 0

    return sum

'''7. this is just a for loop for all the codes in a particular list'''

def get_expected_value_for_all(permutations_list,n):
    total = {}
    for thing in permutations_list:
        input_code = ''.join(thing)
        x = get_prob_dist(input_code,permutations_list,n)
        expected_value = calculate_expected_information(x)
        total[input_code] = expected_value
    
    return total

''' this is to help us find a key for a given value'''
def find_key(dictionary, value):
    matching_keys = [key for key, val in dictionary.items() if val == value]
    return matching_keys if matching_keys else None



''' 8. this computes the the expected values for all the guesses in a list and then selects the one with the largest expected value'''
def find_next_guess(possible_answers,n):
    total_guesses = get_expected_value_for_all(possible_answers,n)
    largest_expected_Value = max(total_guesses.values())
    guess = find_key(total_guesses, largest_expected_Value)
    print(f"this is the largest expected value: {largest_expected_Value} ")

    return guess[0]



'''9. remove unwanted guesses, by computing the clue and seeing if the clue is the same'''

def remove_unwanted(possible_answers,guess,clue):
    other_list = []
    for thing in possible_answers:
        clue_output = Clues(''.join(thing),guess[0])
        if clue == clue_output:
            other_list.append(thing)
    return other_list






'''10. displays prior guesses'''

def display_guess(guess_code):
    for thing in guess_code:
        a = " ".join(thing)
        
        b = guess_code[thing]
        print(a, '|', b )


'''
Check Guess and select guess
'''

def Guess(total_guesses,possible_answers,length_of_code):
    Guess = ''
    y = False
    while not(len(Guess)==length_of_code and y):  
        print('Numbers to Choose from: ')
        print('1 2 3 4 5 6\n')

        g = []
        Guess = find_next_guess(possible_answers,length_of_code)
        if len(Guess) != 4:
            print('You entered a code that is not 4 digits long')
            print('Please Try Again')
        for item in Guess:
            g.append(item)
        K_guess = tuple(g)
        if K_guess in total_guesses:
            y = False
            print('You have already guesses this Code \n')
            print('Please Try Again')
        else:
            y = True
    h = [Guess,K_guess]
    return h




In [66]:


def play_game():
    total_guesses = {}
    
    print('')

    print("Let's get started! \n")

    print("Hmmm... I am thinking of a code \n")

    
    # Ask the user for the number of numbers they can select from
    max_number = int(input("Enter the number of numbers you can select from (e.g., 6 for numbers 1-6): "))

    # Ask the user for the length of the code
    code_length = int(input("Enter the length of the code: "))
    max_num_of_guesses = int(input("Enter max number of guesses: "))

    possible_answers = generate_permutations_with_duplicates(max_number, code_length)
    code = choose_code(max_number,code_length)
    

    print('Ok! I got one.\n')
 
    guess_num = 0
    while guess_num < max_num_of_guesses:      
        print(len(possible_answers))
        guess = Guess(total_guesses,possible_answers,code_length)
        possible_answers.remove(guess[1])
        clue = Clues(guess[0],code)
        possible_answers = remove_unwanted(possible_answers,guess,clue)
        
        
        total_guesses[guess[1]] = clue
        guess_num += 1

        right_answer = (0,code_length)
        if clue == right_answer:
            print('Congratulations That was my code')
          
            display_guess(total_guesses)
          
            print('you Guessed it in ', guess_num, ' guesses!')
            break
        else:
            print("Here are your previous Guesses and clues: \n")
            display_guess(total_guesses)
        
            print('You have ', 12 - guess_num, " guesses left \n")
       
            
            continue
    
    if not(clue ==right_answer) :
        print('You ran out of guesses')
        print('My code was: ', code)


x = play_game()





Let's get started! 

Hmmm... I am thinking of a code 



Enter the number of numbers you can select from (e.g., 6 for numbers 1-6):  7
Enter the length of the code:  4
Enter max number of guesses:  12


Ok! I got one.

2401
Numbers to Choose from: 
1 2 3 4 5 6

this is the largest expected value: 2.982349855197503 
Here are your previous Guesses and clues: 

1 2 3 4 | (2, 1)
You have  11  guesses left 

168
Numbers to Choose from: 
1 2 3 4 5 6

this is the largest expected value: 2.734277363241244 
Here are your previous Guesses and clues: 

1 2 3 4 | (2, 1)
1 3 4 5 | (1, 1)
You have  10  guesses left 

32
Numbers to Choose from: 
1 2 3 4 5 6

this is the largest expected value: 3.077319142645923 
Here are your previous Guesses and clues: 

1 2 3 4 | (2, 1)
1 3 4 5 | (1, 1)
3 2 4 2 | (2, 1)
You have  9  guesses left 

3
Numbers to Choose from: 
1 2 3 4 5 6

this is the largest expected value: 1.5849625007211559 
Congratulations That was my code
1 2 3 4 | (2, 1)
1 3 4 5 | (1, 1)
3 2 4 2 | (2, 1)
3 3 2 4 | (0, 4)
you Guessed it in  4  guesses!
